In [41]:
import pandas as pd
import re

In [42]:
nsmc_train_df = pd.read_csv('ratings_train.txt', sep='\t', encoding='utf8', engine='python')
nsmc_train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [43]:
nsmc_train_df = nsmc_train_df[nsmc_train_df['document'].notnull()]
nsmc_train_df['document'] = nsmc_train_df['document'].apply(lambda x : re.sub(r'[^ㄱ-ㅣ가-힣]', " ", x))
nsmc_train_df['document']

C:\Users\admin\AppData\Local\Temp\ipykernel_13736\3815305855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nsmc_train_df['document'] = nsmc_train_df['document'].apply(lambda x : re.sub(r'[^ㄱ-ㅣ가-힣]', " ", x))


0                                       아 더빙   진짜 짜증나네요 목소리
1                         흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼   솔직히 재미는 없다  평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지   소는 뭔죄인가  
149996                                        평점이 너무 낮아서   
149997                      이게 뭐요  한국인은 거들먹거리고 필리핀 혼혈은 착하다 
149998                          청춘 영화의 최고봉 방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

In [44]:
nsmc_test_df = pd.read_csv('ratings_test.txt', sep='\t', encoding='utf8', engine='python')
nsmc_test_df = nsmc_test_df[nsmc_test_df['document'].notnull()]
nsmc_test_df['document'] = nsmc_test_df['document'].apply(lambda x : re.sub(r'[^ㄱ-ㅣ가-힣]', " ", x))
nsmc_test_df['document']

0                                                      굳 ㅋ
1                                                         
2                   뭐야 이 평점들은     나쁘진 않지만   점 짜리는 더더욱 아니잖아
3                         지루하지는 않은데 완전 막장임    돈주고 보기에는    
4          만 아니었어도 별 다섯 개 줬을텐데   왜   로 나와서 제 심기를 불편하게 하죠  
                               ...                        
49995            오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996         의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따     
49997                   그림도 좋고 완성도도 높았지만    보는 내내 불안하게 만든다
49998       절대 봐서는 안 될 영화   재미도 없고 기분만 잡치고   한 세트장에서 다 해먹네
49999                                           마무리는 또 왜이래
Name: document, Length: 49997, dtype: object

In [45]:
from konlpy.tag import Okt
okt =Okt()
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [46]:
# TF-IDF 기반 피처 벡터 생성 :
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(nsmc_train_df['document'])
nsmc_train_tfidf = tfidf.transform(nsmc_train_df['document'])

nsmc_train_tfidf

c:\Users\admin\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<149995x115703 sparse matrix of type '<class 'numpy.float64'>'
	with 2712770 stored elements in Compressed Sparse Row format>

In [47]:
from sklearn.linear_model import LogisticRegression
SA_lr = LogisticRegression(random_state=0)
SA_lr.fit(nsmc_train_tfidf,nsmc_train_df['label'])

LogisticRegression(random_state=0)

In [54]:
from sklearn.model_selection import GridSearchCV
params = {'C' : [1, 3, 3.5, 4, 4.5, 5]}
# cv : 교차검증을 위한 fold 횟수 // verbose : 0(default) 메시지 출력안함, verbose : 1 간단한 메시지 출력
# 																verbose : 2 하이퍼 파라미터별 메시지 출력
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)
SA_lr_grid_cv.fit(nsmc_train_tfidf,nsmc_train_df['label'])
SA_lr_best = SA_lr_grid_cv.best_estimator_

Fitting 3 folds for each of 6 candidates, totalling 18 fits


c:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [55]:
# 평가용 데이터의 피처 벡터화 :
nsmc_test_tfidf = tfidf.transform(nsmc_test_df['document'])
test_predict = SA_lr_best.predict(nsmc_test_tfidf)

from sklearn.metrics import accuracy_score
print('감성 분석 정확도 : ', round(accuracy_score(nsmc_test_df['label'],test_predict),3))


감성 분석 정확도 :  0.858


In [82]:
st = input('감성 분석할 문장입력 >> ')
print(st)

너는 봄날


In [83]:
# 0) 입력 텍스트에 대한 전처리 수행
st = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(st)
print(st)
st = [" ".join(st)]
print(st)

['너는', '봄날']
['너는 봄날']


In [70]:
# 1) 입력테스트의 피처 백터화
st_tfidf = tfidf.transform(st)
# 2) 최적 감성분석 모델에 적용하여 감성 분석 평가
st_predict = SA_lr_best.predict(st_tfidf)
st_predict

array([0], dtype=int64)

In [71]:
# 3) 예측값 출력하기
if(st_predict == 0):
    print(st,"==> 부정 감성")
else:
    print(st,"==> 긍정 감성")

['너 는   바 보'] ==> 부정 감성
